In [1]:
import os
# 【硬性要求】加载环境变量 (Embeddings 调用需要 OpenAI API Key)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_openai import OpenAIEmbeddings
# 【规范更新】从专门的 chroma 包导入，这是当前官方的推荐做法
from langchain_chroma import Chroma

# ==========================================
# 1. 初始化嵌入模型 (Initialization)
# ==========================================
# 确保与之后查询时使用的是同一个模型坐标系
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# ==========================================
# 2. 准备语料并建立向量库 (Indexing Phase)
# ==========================================
# 模拟体育新闻大纲 (两个板球相关，两个足球相关)
docs = [
    "Thrilling Finale Awaits: The Countdown to the Cricket World Cup Championship",
    "Global Giants Clash: Football World Cup Semi-Finals Set the Stage for Epic Showdowns",
    "Record Crowds and Unforgettable Moments: Highlights from the Cricket World Cup",
    "From Underdogs to Contenders: Football World Cup Surprises and Breakout Stars"
]

print("正在调用 OpenAI 接口将文本向量化，并存入 Chroma 向量数据库 (这可能需要几秒钟)...")
# from_texts 实际上是一个快捷工厂方法：
# 它在后台执行了: 1. 调用 embeddings 模型将 docs 转换成向量 2. 将文本和对应的向量塞进 Chroma 实例中
vectorstore = Chroma.from_texts(texts=docs, embedding=embeddings)
print("✅ 向量库构建完成！\n")

# ==========================================
# 3. 执行语义搜索测试 (Retrieval Phase)
# ==========================================
if __name__ == "__main__":
    # 测试用例 A：板球运动员
    query_a = "Rohit Sharma"
    print(f"=== 🔍 搜索关键词: '{query_a}' ===")
    # k=2: 代表 Top-K 参数，只检索最相似的前 2 个文档块
    results_a = vectorstore.similarity_search(query_a, k=2)
    for i, res in enumerate(results_a):
        print(f"匹配 {i+1}: {res.page_content}")
        
    print("\n" + "-"*50 + "\n")
    
    # 测试用例 B：足球运动员
    query_b = "Lionel Messi"
    print(f"=== 🔍 搜索关键词: '{query_b}' ===")
    results_b = vectorstore.similarity_search(query_b, k=2)
    for i, res in enumerate(results_b):
        print(f"匹配 {i+1}: {res.page_content}")

正在调用 OpenAI 接口将文本向量化，并存入 Chroma 向量数据库 (这可能需要几秒钟)...
✅ 向量库构建完成！

=== 🔍 搜索关键词: 'Rohit Sharma' ===
匹配 1: Record Crowds and Unforgettable Moments: Highlights from the Cricket World Cup
匹配 2: Thrilling Finale Awaits: The Countdown to the Cricket World Cup Championship

--------------------------------------------------

=== 🔍 搜索关键词: 'Lionel Messi' ===
匹配 1: From Underdogs to Contenders: Football World Cup Surprises and Breakout Stars
匹配 2: Global Giants Clash: Football World Cup Semi-Finals Set the Stage for Epic Showdowns
